In [23]:
import polars as pl
import pandera as pa

In [62]:
stops = pl.read_csv('stops.txt')
stops.head(3)

stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,platform_code
str,str,str,str,f64,f64,str,str,i64,str
"""1_01""",null,"""米沢スキー場前""",null,37.863098,140.231997,"""1_01""",null,0,null
"""10_01""",null,"""工業団地入口""",null,37.895684,140.162863,"""10_01""",null,0,null
"""10_02""",null,"""工業団地入口""",null,37.895615,140.163126,"""10_02""",null,0,null


In [63]:
stops = stops.with_columns(
    pl.lit(None).cast(str).alias('parent_station')
).select(
    pl.col('stop_id').cast(str),
    pl.col('stop_name').cast(str),
    pl.col('parent_station'),
    pl.col('platform_code').cast(str),
    pl.col('stop_lat').cast(pl.Float64),
    pl.col('stop_lon').cast(pl.Float64),
)
stops.write_parquet('stops.parquet')

In [26]:
stop_times = pl.read_csv('stop_times.txt')
stop_times.head(3)

trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled
str,str,str,str,i64,str,i64,i64,i64
"""1全日夏期_07時00分_系…","""07:00:00""","""07:00:00""","""26_01""",1,"""米沢駅前""",0,1,0
"""1全日夏期_07時00分_系…","""07:01:00""","""07:01:00""","""38_01""",2,"""米沢駅前""",0,0,0
"""1全日夏期_07時00分_系…","""07:02:00""","""07:02:00""","""39_01""",3,"""米沢駅前""",0,0,0


In [27]:
stop_times = stop_times.select(
    pl.col('trip_id').cast(str),
    pl.col('stop_sequence').cast(pl.Int64),
    pl.col('stop_id').cast(str),
    pl.col('arrival_time').str.replace_all(':','').cast(pl.Int32),
    pl.col('departure_time').str.replace_all(':','').cast(pl.Int32),
    pl.col('pickup_type').cast(pl.Int32),
    pl.col('drop_off_type').cast(pl.Int32),
)
stop_times.write_parquet('stop_times.parquet')

In [28]:
trips = pl.read_csv('trips.txt')
trips.head(3)

route_id,service_id,trip_id,trip_headsign,direction_id,block_id,trip_short_name,shape_id,jp_trip_desc
str,str,str,str,i64,str,str,str,str
"""10B""","""平日土曜夏期""","""3平日土曜夏期_07時25分…","""米沢市役所前""",0,null,null,null,null
"""10B""","""平日土曜冬期""","""4平日土曜冬期_07時05分…","""米沢市役所前""",0,null,null,null,null
"""10B""","""平日土曜""","""5平日土曜_09時00分_系…","""米沢市役所前""",0,null,null,null,null


In [57]:
calendar = pl.read_csv('calendar_dates.txt')
calendar.head(3)

service_id,date,exception_type
str,i64,i64
"""全日冬期""",20220401,2
"""全日冬期""",20220402,2
"""全日冬期""",20220403,2


In [60]:
calendar = calendar.groupby(
    "date", maintain_order=True
).agg(
    [pl.col('service_id')]
).select(
    pl.col('date').cast(str).str.to_date('%Y%m%d').alias('calendar_date'),
    pl.col('service_id').alias('service_ids'),
)
calendar.write_parquet('calendar_dates.parquet')

In [61]:
calendar

calendar_date,service_ids
date,list[str]
2022-04-01,"[""全日冬期"", ""平日土曜冬期""]"
2022-04-02,"[""全日冬期"", ""平日土曜冬期""]"
2022-04-03,"[""全日冬期"", ""平日土曜冬期""]"
2022-04-04,"[""全日冬期"", ""平日土曜冬期""]"
2022-04-05,"[""全日冬期"", ""平日土曜冬期""]"
2022-04-06,"[""全日冬期"", ""平日土曜冬期""]"
2022-04-07,"[""全日冬期"", ""平日土曜冬期""]"
2022-04-08,"[""全日冬期"", ""平日土曜冬期""]"
2022-04-09,"[""全日冬期"", ""平日土曜冬期""]"


In [37]:
transfers = pl.read_csv('transfers.txt')
transfers.head(3)

from_stop_id,to_stop_id,transfer_type,min_transfer_time
str,str,str,str


In [38]:
transfers = transfers.select(
    pl.col('from_stop_id').cast(str),
    pl.col('to_stop_id').cast(str),
    pl.col('transfer_type').cast(pl.Int32),
    pl.col('min_transfer_time').cast(pl.Int32)
)
transfers.write_parquet('transfers.parquet')